# Bien choisir les ressources
Il y a plusieurs ressources informatiques qui vous sont disponibles à
l'Alliance de Recherche numérique du Canada :
* Calcul haute performance
  * Béluga, Cedar, Graham, Niagara
* Stockage
  * Temporaire, projet, *nearline*, dépôt de données de recherche
* Infonuagique
  * Arbutus, Béluga, Cedar, Graham

Le principal but de ce chapitre est de vous permettre d'analyser
vos besoins en **ressources de calcul haut-performance**, et ce,
dans le but de choisir les ressources nécessaires pour votre projet.

## Rappel - le calcul haute performance
* Lorsqu'il est question de lancer une **grande quantité de calculs**
(séquentiels ou parallèles) ou de **traitements de données**,
l'utilisation d'une grappe de calcul haute performance devient nécessaire.
* Puisque les **ressources sont partagées** et en grande demande,
chaque tâche doit être soumise à un ordonnanceur de tâches.
* Il devient donc nécessaire **d'estimer à l'avance les ressources**
qui seront réservées lors de l'exécution d'un calcul.

Dans cette section, on cherche à évaluer le comportement d'une tâche de calcul
afin d'estimer et d'optimiser les ressources à réserver sur la grappe de calcul.

### Objectif - construction d'un script de tâche Slurm
Un script de tâche pour [l'ordonnanceur Slurm](https://slurm.schedmd.com/documentation.html) est typiquement un script Bash dans lequel on retrouve :
* Le [shebang](https://fr.wikipedia.org/wiki/Shebang) en toute première ligne. Par exemple : `#!/bin/bash`
* Les options `#SBATCH` en entête pour les besoins de la tâche. Les options en entête seront lues par la commande de soumission de tâche [`sbatch`](https://slurm.schedmd.com/sbatch.html)
* [Chargement des modules](https://docs.computecanada.ca/wiki/Utiliser_des_modules) requis
* Les commandes Bash qui seront exécutées automatiquement sur des processeurs réservés pour la tâche

Par exemple : [`scripts/mpi_allo.sh`](https://github.com/calculquebec/utilisation-serveurs-calcul/blob/main/scripts/mpi_allo.sh)

```
#!/bin/bash
#SBATCH --ntasks=10
#SBATCH --mem-per-cpu=1000M
#SBATCH --time=0-00:10

mpirun printenv HOSTNAME OMPI_COMM_WORLD_RANK OMPI_COMM_WORLD_SIZE
```

Notre documentation à cet effet débute à la page : [Exécuter des tâches](https://docs.computecanada.ca/wiki/Running_jobs/fr)

## Analyse des calculs à petite échelle
Lorsqu'une tâche de calcul est en cours sur votre ordinateur, vous pouvez surveiller différentes métriques :
* Utilisation CPU
* Mémoire résidente (réellement utilisée)
* Mémoire virtuelle (allouée)
* Les accès au stockage

### Sous Windows
* [Gestionnaire des tâches Windows](https://fr.wikipedia.org/wiki/Gestionnaire_des_t%C3%A2ches_Windows)
* Pour le faire afficher :
  * Raccourcis clavier Ctrl+Alt+Suppr ou cliquer sur la barre des tâches avec le bouton droit
  * Cliquer sur l'option *Gestionnaire des tâches*

![Aperçu du gestionnaire des tâches Windows](images/win-task-manager.png)

### Sous Mac OS
* [Moniteur d’activité](https://support.apple.com/fr-ca/guide/activity-monitor/actmntr1001/mac)
* Pour le faire afficher :
  * Démarrer l'application à partir des *Applications et Utilitaires* de Mac OS
  * Sinon, utiliser le raccourcis clavier Commande+Espace et taper les premières lettres de "Moniteur d'activité" pour pouvoir sélectionner cette application

![Aperçu du moniteur d'activité Mac OS](https://help.apple.com/assets/5FDCF1894EB74318147EC0CF/5FDCF18A4EB74318147EC0D6/fr_CA/ad6337d66061aa27122e75521960fc5a.png)

### Sous Linux
Dans un terminal Unix, on peut utiliser :
* La [commande `top`](https://man7.org/linux/man-pages/man1/top.1.html) (`q` pour quitter)

![Capture de top](images/linux-top.png)

* La [commande `htop`](https://man7.org/linux/man-pages/man1/htop.1.html) (`q` pour quitter)

![Capture de htop](images/linux-htop.png)

### Sur la grappe de calcul directement
Avec votre accès par défaut, vous avez un compte de calcul `def-*` de base
vous permettant de lancer des tâches de test.

#### Vérifier l'utilisation du CPU et de la mémoire
```
ssh login1
...
```

Avec la commande `seff`, on peut obtenir un court rapport d'exécution de tâche.
Ce rapport inclut une mesure du temps écoulé, une mesure du temps CPU
et une mesure de consommation maximale de la mémoire-vive.
Des valeurs d'efficacité en pourcentages sont données pour les cycles CPU
et la mémoire-vives en fonction des quantités réservées.

#### Vérifier l'utilisation du GPU
```
bash scripts/installer/cuda-samples.sh
salloc --gres=gpu:1 --mem=2000M --time=0-00:05
bash scripts/cuda-matmul.sh &> /dev/null &
```

* Pour Windows et Mac OS, il existe des outils propriétaires permettant de visualiser en temps réel l'utilisation du GPU. Veuillez vous référer aux sites Web du manufacturier de votre GPU pour les détails
* Sous Linux, il y a d'abord la [commande `nvidia-smi`](https://developer.nvidia.com/nvidia-system-management-interface)

![Capture nvidia-smi](images/nvidia-smi.png)

* Il existe aussi un projet [`nvtop`](https://github.com/Syllo/nvtop) permettant de visualiser l'utilisation d'un ou plusieurs GPUs dans un terminal

![Capture nvtop](https://raw.githubusercontent.com/Syllo/nvtop/master/screenshot/NVTOP_ex1.png)

#### Comparer la vitesse CPU vs GPU
Si votre application ne fait pas bon usage d'un GPU ou n'est pas du tout conçue pour fonctionner avec un accélérateur, il n'y a aucun avantage à soumettre une tâche en demandant un ou plusieurs GPUs.
* Avant d'utiliser massivement les GPUs d'une grappe de calcul, il faut tout d'abord que l'application puisse démontrer une "bonne performance" en utilisant plusieurs processeurs en parallèle.
 * La métrique à noter et à comparer est le **temps écoulé**, et non le temps CPU.
 * **Accélération** = (temps avec un processeur) / (temps avec plusieurs processeurs)
 * **Efficacité** = (Accélération) / (nombre de processeurs)
* Le coût d'un noeud GPU étant de quatre à cinq fois supérieur à celui d'un noeud régulier, l'utilisation d'un seul GPU doit permettre une accélération d'au moins quatre fois (4x) la vitesse de huit (8) à dix (10) processeurs.
 * **Accélération** = (temps avec 8 à 10 processeurs) / (temps avec un GPU)

#### Exercice - Calcul d'accélération et d'efficacité
Soumission d'une tâche séquentielle et d'une tâche parallèle :
* **Si ce n'est pas déjà fait**, télécharger et compiler le programme N-Body en exécutant :
```
bash scripts/installer/nbody.sh
```
* Explorer le script de tâche [`scripts/nbody-openmp.sh`](https://github.com/calculquebec/utilisation-serveurs-calcul/blob/main/scripts/nbody-openmp.sh)
```
cat scripts/nbody-openmp.sh
```
* Soumettre ce script à l'ordonnanceur en demandant un (1) processeur par tâche :
```
sbatch --cpus-per-task=1 scripts/nbody-openmp.sh
```
* Soumettre à nouveau ce script à l'ordonnanceur en demandant quatre (4) processeurs par tâche :
```
sbatch --cpus-per-task=4 scripts/nbody-openmp.sh
```
* Attendre que les deux tâches se terminent (après les états `PD` et `R`) :
```
squeue -u $USER  # Environ aux 30 secondes
```

Analyse des résultats :
* Les résultats seront dans les fichiers `slurm-JOBID.out`
  * Pour une exécution valide, tous les fichiers `*.mol` doivent être "`OK`"
```
grep .mol slurm-*.out
```
  * Le temps écoulé est donné à la dernière ligne des fichiers `slurm-JOBID.out`
```
tail -n 1 slurm-*.out
```
* Selon les résultats obtenus ci-dessus, une analyse d'accélération et
d'efficacité peut être faite à l'aide du script 
[`scripts/calc-acc-eff.sh`](https://github.com/calculquebec/utilisation-serveurs-calcul/blob/main/scripts/calc-acc-eff.sh)
```
cat scripts/calc-acc-eff.sh
```
* Exemple d'utilisation : avec 61.8 secondes pour 1 processeur et 16.8 secondes pour 4 processeurs :
```
bash scripts/calc-acc-eff.sh 1:61.8 4:16.8
```
* Une efficacité parfaite serait de 1.0 (ou 100%) :
```
Metrique,Valeur
acc_lineaire,4.00000000000000000000
acceleration,3.67857142857142857142
efficacite,.91964285714285714285
```

## Extrapoler les ressources nécessaires
#### Efficacité cible du calcul parallèle
Une efficacité de 80%, voire 90%, devrait être un seuil minimal pour les tâches parallèles. Il reste donc à **déterminer le nombre maximal de processeurs** pour notre application.

Nous allons utiliser la [loi d'Amdahl](https://fr.wikipedia.org/wiki/Loi_d%27Amdahl). Voici quelques définitions :
* $T_s$: temps requis pour une exécution avec un seul processeur (donc 100% séquentiel)
* $P$ : fraction de $T_s$ correspondant à des opérations **parallèles**, donc **divisible** par $n$ processeurs.
* $S$ : fraction de $T_s$ correspondant à des opérations **séquentielles**, donc **non-divisible** par $n$ processeurs.
 * Exemples d'opérations séquentielles : lecture-écriture d'un fichier, communications, synchronisation, etc.
* Dans ce modèle, $P + S = 1$, donc $S = 1 - P$

$$T_p(n) = T_s * \left(S + \frac{P}{n}\right) = T_s * \left(1 - P + \frac{P}{n}\right)$$

* De là, on peut redéfinir l'accélération $A(n)$ selon $n$ processeurs et isoler $P$ pour éventuellement le calculer :

$$A(n) = \frac{T_p(1)}{T_p(n)} = \frac{T_s * \left(1 - P + \frac{P}{1}\right)}{T_s * \left(1 - P + \frac{P}{n}\right)} = \frac{1 - P + P}{1 - P + \frac{P}{n}} = \frac{1}{1 - P + \frac{P}{n}}$$

$$\frac{1}{A(n)} = 1 - P + \frac{P}{n} = 1 - P * \left(1 - \frac{1}{n}\right) \implies P * \left(1 - \frac{1}{n}\right) = 1 - \frac{1}{A(n)}$$

$$P = \frac{1 - \frac{1}{A(n)}}{1 - \frac{1}{n}}$$

* Pour finalement imposer une efficacité $E(n)$ minimale $e$ afin de calculer le $n$ maximal :

$$E(n) = \frac{A(n)}{n} \geq e \implies A(n) \geq e * n \implies \frac{1}{1 - P + \frac{P}{n}} \geq e * n$$

$$1 \geq e * n * \left(1 - P + \frac{P}{n} \right) = e * (1 - P) * n + e * P$$

$$1 - e * P \geq (e - e * P) * n \implies \frac{1 - e * P}{e - e * P} \geq n$$

$$n \leq \frac{\frac{1}{e} - P}{1 - P}$$

* De là, c'est possible de calculer $n$ lorsque $e = 0.8$, par exemple.

**Exercice - Taille maximale d'une tâche parallèle**

* Explorer le script [`scripts/calc-n-max.sh`](https://github.com/calculquebec/utilisation-serveurs-calcul/blob/main/scripts/calc-n-max.sh)
```
cat scripts/calc-n-max.sh
```
* **Important** - ce script est conçu pour recevoir le résultat redirigé du script
[`scripts/calc-acc-eff.sh`](https://github.com/calculquebec/utilisation-serveurs-calcul/blob/main/scripts/calc-acc-eff.sh) dans son canal STDIN
  * En cas de blocage du terminal, faites Ctrl+C
* Réutilisons les résultats du précédent exercice pour calculer `n_max`, soit le nombre maximal suggéré de processeurs pour une efficacité minimale de 80%. Par exemple :
```
bash scripts/calc-acc-eff.sh 1:61.8 4:16.8 | bash scripts/calc-n-max.sh
```
* On note que le calcul de `n_max` peut donner une valeur qu'il faut arrondir à la baisse, car l'efficacité diminue en fonction du nombre de processeurs :
```
Metrique,Valeur
P,0.970874
n_max,9.58333
n_max_entier,9
```

#### Taille des données et nombre de fichiers à traiter
Pour un calcul donné, il y a deux métriques de stockage à considérer :
* La **quantité** totale en octets (ou Go)
* Le **nombre** total de fichiers

Pour obtenir ces informations :
* **Sous Windows** : dans l'explorateur Windows (raccourcis clavier : Windows + E)
  * Sélectionner un dossier ou plusieurs fichiers
  * Bouton droit de la souris -> *Propriétés*

![Windows data properties](images/win-data-size.png)

* **Sous Mac OS** : dans *Finder*
  * Sélectionner un dossier ou plusieurs fichiers
  * Bouton droit de la souris -> *Get Info*
  * Autrement : avec l'affichage *Par liste*
    * [Activer *Calculer toutes les tailles*](https://www.solutionenligne.org/comment-afficher-taille-dossiers-fichiers-dans-finder-mac-os/)

* **Sous Linux** :
  * L'environnement graphique peut offrir le même genre d'outils, mais tout dépend de la distribution et du bureau.
  * La commande `du -bs DOSSIER` (`b` : taille apparente en octets, `s` : somme totale) calcule récursivement et affiche la taille totale en octets. La taille apparente est celle qui importe lors d'un transfert ou d'une sauvegarde de données.
  * La commande `find DOSSIER | wc -l` compte récursivement et affiche le nombre de fichiers et de sous-dossiers.

#### Stockage en mémoire selon les types de base
En ayant une idée de la taille des données à traiter, il devient aussi possible d'estimer la place que les données peuvent prendre en mémoire-vive.

* Dans un fichier texte, **chaque caractère** prend de un (1) à deux (2) octets, en moyenne.
Cependant, pour certaines langues, l'encodage
[UTF-8](https://fr.wikipedia.org/wiki/UTF-8#Description)
peut se rendre jusqu'à quatre (4) octets par caractère.
Pour les langues latines et germaniques, on peut considérer **deux (2) octets** par caractère.
Par exemple, dans une session Python :

```
python
```

```
>>> import sys
>>> euro = ""
>>> sys.getsizeof(euro)
49
>>> euro = "Euro"
>>> sys.getsizeof(euro)
53
>>> euro = "€"
>>> sys.getsizeof(euro)
76
>>> euro *= 2  # Donc "€€"
>>> sys.getsizeof(euro)
78
```
* Note : pour sortir de la session Python : Ctrl+D ou `quit()`

* Les **nombres entiers** prennent typiquement 1 octet (8 bits), 2 octets (16 bits), 4 octets (32 bits) ou 8 octets (64 bits) chacun. Tout dépend de la [plage de valeurs souhaitée](https://en.wikipedia.org/wiki/C_data_types#Main_types):
  * 1 octet : 256 valeurs de 0 à 255, ou de -128 à 127
  * 2 octets : ~65 milles valeurs de $0$ à $(2^{16}-1)$, ou de $-2^{15}$ à $(2^{15}-1)$
  * 4 octets : ~4 milliards de valeurs de 0 à $(2^{32}-1)$ ou de $-2^{31}$ à $(2^{31}-1)$
  * 8 octets : ~18 trillions de valeurs de 0 à $(2^{64}-1)$ ou de $-2^{63}$ à $(2^{63}-1)$

```
module load python scipy-stack
python
```

```
>>> import sys
>>> import numpy as np
>>> cube = np.zeros((100,100,100), dtype=np.int64)
>>> sys.getsizeof(cube)
8000136
>>> cube = np.zeros((100,100,100), dtype=np.int32)
>>> sys.getsizeof(cube)
4000136
>>> np.iinfo(np.int16)
iinfo(min=-32768, max=32767, dtype=int16)
```

* Les **nombres à virgule flottante** prennent typiquement 4 octets (32 bits) ou 8 octets (64 bits) chacun, mais on voit de plus en plus [différents types de données à 2 octets (16 bits)](https://en.wikipedia.org/wiki/Bfloat16_floating-point_format) dans des applications d'apprentissage-machine. Il se peut néanmoins que les données soient initialement en simple ou double précision:
  * [simple précision](https://en.wikipedia.org/wiki/Single-precision_floating-point_format) : 4 octets, une résolution de 23 bits (~7 décimales), une échelle de 8 bits (${10}^{-38}$ à ${10}^{38}$)
  * [double précision](https://en.wikipedia.org/wiki/Double-precision_floating-point_format) : 8 octets, une résolution de 52 bits (~16 décimales), une échelle de 11 bits (${10}^{-308}$ à ${10}^{308}$)

```
>>> cube = np.ndarray((100,100,100), dtype=np.float32)
>>> cube[0,0,0] = np.pi
>>> print(cube[0,0,0])
3.1415927
>>> print(cube[0,0,0], np.pi)
3.1415927 3.141592653589793
```

* Certains langages utilisent systématiquement **8 octets** (64 bits) par nombre.
* [Certains compilateurs et certaines bibliothèques](https://en.wikipedia.org/wiki/Quadruple-precision_floating-point_format) peuvent calculer des valeurs représentées avec 128 bits [ou plus](https://gmplib.org/).
* Pour les nombres complexes, on multiplie l'espace mémoire par deux (2).

Exemple de calcul de l'espace-mémoire :
```
>>> nb_matrices = 3        # Trois matrices C = prod_mat(A, B)
>>> taille = 25000         # Matrices carrées
>>> octets_par_nombre = 8  # Double précision
>>> memoire = nb_matrices * taille*taille * octets_par_nombre
>>> memoire / 1000**3
```

#### La complexité des algorithmes
La question qui se pose : en augmentant la ou les dimensions du problème, quelles devraient être la durée du calcul et la consommation en mémoire-vive?
Une [analyse de la complexité de l'algorithme principal](https://fr.wikipedia.org/wiki/Analyse_de_la_complexit%C3%A9_des_algorithmes)
permettrait de connaître l'ordre $O$ du calcul en fonction de la taille $n$ des données :

* $O(n)$: proportionnel à $n$
* $O(n*m)$: représente un calcul à deux (2) dimensions indépendantes
* $O(n^3)$: calcul d'ordre cubique
* $O(n*m*k^2)$: par exemple, un filtre de taille $k*k$ sur une image $n*m$
* $O(n*log(n))$: typique de certains
[algorithmes de tri](https://fr.wikipedia.org/wiki/Algorithme_de_tri#Comparaison_des_algorithmes)
où il y a $n$ éléments à trier en $log_2(n)$ étapes.

Une analyse détaillée du code (s'il est disponible) n'est pas nécessaire pour déterminer le type de calcul qui est fait.
* Vous pouvez vous inspirer des données en entrées pour deviner l'ordre du calcul principal. 
* Vous pouvez mesurer le temps d'exécution en fonction de la taille du problème. En extrapolant les résultats, il serait possible de prévoir le comportement du programme sur une grappe de calcul.

#### Exercice - Complexité de l'algorithme
Dans cet exercice, il est question d'inverser une matrice de valeurs aléatoires de taille `n*n` :
* Soumettre le script [`scripts/inv-mat.sh`](https://github.com/calculquebec/utilisation-serveurs-calcul/blob/main/scripts/inv-mat.sh) avec la commande `sbatch` :

```
sbatch scripts/inv-mat.sh
```

* Explorer le script Python [`scripts/inv-mat.py`](https://github.com/calculquebec/utilisation-serveurs-calcul/blob/main/scripts/inv-mat.py) :

```
less scripts/inv-mat.py  # q pour quitter
```

* Suivre l'évolution du calcul avec `squeue -u $USER`, environ aux 30 secondes
* Le résultat sera sauvegardé dans le fichier `temps_inv.csv`
* Analyse avec Python, Pandas et Numpy dans le script [`scripts/inv-mat-pred.py`](https://github.com/calculquebec/utilisation-serveurs-calcul/blob/main/scripts/inv-mat-pred.py) :

```
module load gcc python scipy-stack

cat scripts/inv-mat-pred.py
python scripts/inv-mat-pred.py temps_inv.csv
```

## Principales différences entre les grappes de calcul
* À propos des grappes :

| | [Béluga](https://docs.computecanada.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.computecanada.ca/wiki/Cedar/fr) | [Graham](https://docs.computecanada.ca/wiki/Graham/fr) | [Narval](https://docs.computecanada.ca/wiki/Narval) | [Niagara](https://docs.computecanada.ca/wiki/Niagara/fr) |
|-----------------------:|:---------:|:---------:|:---------:|:------------:|:----------:|
| **Mise en production** | Mars 2019 | Juin 2017 | Juin 2017 | Octobre 2021 | Avril 2018 |
|              **Ville** | Montréal  |  Burnaby  | Waterloo  |   Montréal   |   Toronto  |
|           **Province** |  Québec   |    C.-B.  |  Ontario  |    Québec    |   Ontario  |

* Nombre de processeurs (coeurs CPU) selon le cas :

| Processeur Intel/AMD  | [Béluga](https://docs.computecanada.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.computecanada.ca/wiki/Cedar/fr) | [Graham](https://docs.computecanada.ca/wiki/Graham/fr) | [Narval](https://docs.computecanada.ca/wiki/Narval) | [Niagara](https://docs.computecanada.ca/wiki/Niagara/fr) |
|----------------------:|:--------:|:--------:|:--------:|:---------:|:---------:|
|      Broadwell (avx2) |          | 724 * 32 | 983 * 32 |           |           |
|      Skylake (avx512) | 802 * 40 | 640 * 48 |          |           | 1548 * 40 |
| Cascade Lake (avx512) |          | 768 * 48 |  72 * 44 |           |  468 * 40 |
|      EPYC Rome (avx2) |          |          |          | 1142 * 64 |           |

| Mémoire par proc. | [Béluga](https://docs.computecanada.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.computecanada.ca/wiki/Cedar/fr) | [Graham](https://docs.computecanada.ca/wiki/Graham/fr) | [Narval](https://docs.computecanada.ca/wiki/Narval) | [Niagara](https://docs.computecanada.ca/wiki/Niagara/fr) |
|-------:|:-----:|:-----:|:-----:|:-----:|:-----:|
|  2400M |  6400 |       |       |       |       |
|  4000M |       | 86016 | 28896 | 70976 |       |
|  4400M |       |       |  3168 |       |       |
|  4800M | 23560 |       |       |       | 80960 |
|  8000M |       |  3072 |  1792 |       |       |
| 16000M |       |   768 |   768 |       |       |
| 19200M |  2120 |       |       |       |       |
| 32000M |       |       |       |  2112 |       |
| 48000M |       |   768 |   192 |       |       |
| 96000M |       |   128 |       |       |       |

* [Nombre de GPUs](https://docs.computecanada.ca/wiki/Using_GPUs_with_Slurm) selon le cas :

| Accélérateurs | [Béluga](https://docs.computecanada.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.computecanada.ca/wiki/Cedar/fr) | [Graham](https://docs.computecanada.ca/wiki/Graham/fr) | [Mist (Power9)](https://docs.scinet.utoronto.ca/index.php/Mist) | [Narval](https://docs.computecanada.ca/wiki/Narval) |
|----------------:|:---:|:---:|:---:|:---:|:---:|
| NVIDIA P100 12G |     | 456 | 320 |     |     |
| NVIDIA P100 16G |     | 128 |     |     |     |
|   NVIDIA T4 16G |     |     | 144 |     |     |
| NVIDIA V100 16G | 688 |     |  54 |     |     |
| NVIDIA V100 32G |     | 768 |  16 | 216 |     |
| NVIDIA A100 40G |     |     |     |     | 632 |

* Réseau haute-performance et ordonnancement :

| | [Béluga](https://docs.computecanada.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.computecanada.ca/wiki/Cedar/fr) | [Graham](https://docs.computecanada.ca/wiki/Graham/fr) | [Narval](https://docs.computecanada.ca/wiki/Narval) | [Niagara](https://docs.computecanada.ca/wiki/Niagara/fr) |
|------------------------:|:----------:|:-----------:|:----------:|:-----------:|:----------:|
|        Connexion rapide | InfiniBand |   OmniPath  | InfiniBand | InfiniBand  | InfiniBand |
|               Topologie |  En arbre  |   En arbre  |  En arbre  |  En arbre   | DragonFly+ |
|     Taille îlots (proc) | 640 à 1200 | 1024 à 1536 |    1024    | 3072 à 3584 |    17280   |
|     Facteur de blockage |   max 5:1  |   max 2:1   |   max 8:1  |  max 4.7:1  |   max 2:1  |
| Granularité des tâches  | /proc /GPU |  /proc /GPU | /proc /GPU |  /proc /GPU |   /noeud   |
|         Durée maximale  |   7 jours  |   28 jours  |  28 jours  |   7 jours   |   1 jour   |

* Stockage : le tout sera décrit au dernier chapitre.